# IBM Data Science: Capstone Project
### Battle of the Neighborhoods - Week 1

## Arteddy

### Battle of the Neighborhoods -- Business Decision:
### Where to Locate a Professional Services Business: San Francisco SoMa Area or Seattle Pioneer Square?

# Question 1: Description of the Problem and a Discussion of the Background

Address the decision where to locate a professional services business that services a geographical concentration of high tech companies that includes a burgeoning high-tech startup hub.

This project will focus on two Neighbourhoods:
* San Francisco, CA: SoMa (South of Market Street) region
* Seattle, WA: Pioneer Square region

Both of these regions have strong technology company presence, and both are noted for increasing technology startup companies.¶

# Question 2: Description of the Data and How it will be used to solve the Problem

The decision of where to locate a professional services business servicing a high-tech company hub depends on many factors. In this study, I focus on two major sets of factors

   1. Financial impact to the company

   2. Employee quality of life.
    

    

1) Financial impact to the company

    - Commercial Office Real Estate lease and rent costs, based on current commercial office properties available at the time of this study.
    - Business density concentration, utilizing the Foursquare dataset.
    - Business Taxes
    
2) Employee Quality of Life, using the Foursquare dataset.

    Availability of
    - Public Transportation: specify how many subway stations are located in, and the distance of each from the center of each neighborhood.
    - Exercise Facilities: specify how many gyms or exercise facilities are located in.
    - Restaurants: specify how many restaurants are located in.

# Start. from Foursquare, extract venue data for each location


In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests
from bs4 import BeautifulSoup
import csv
import xml
import random          # library for random number generation
print('Library is reade to use.')

Library is reade to use.


In [7]:
import json 
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 

from sklearn.cluster import KMeans

!pip install folium
import folium 
from folium import plugins

print("Library is ready to use.")

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01
Library is ready to use.


In [8]:
ad_sf_sm = 'San Francisco, CA / SoMa Area'

latitude_sf_sm = 37.775316
longitude_sf_sm = -122.419626

ad_se_ps = 'Seattle, WA / Pioneer Square Area'

latitude_se_ps = 47.601954
longitude_se_ps = -122.329204

print('The geographical coordinates of', ad_sf_sm, 'are    : {}  {}'.format(latitude_sf_sm, longitude_sf_sm))
print('The geographical coordinates of', ad_se_ps, 'are: {}  {}'.format(latitude_se_ps , longitude_se_ps))

The geographical coordinates of San Francisco, CA / SoMa Area are    : 37.775316  -122.419626
The geographical coordinates of Seattle, WA / Pioneer Square Area are: 47.601954  -122.329204


In [9]:
# Display map of the region to show the two cities
map_all = folium.Map(location=[(latitude_sf_sm+latitude_se_ps)/2, ((longitude_sf_sm+longitude_se_ps)/2)+5], tiles='Stamen Terrain', zoom_start=5)

folium.Marker(location=[latitude_sf_sm, longitude_sf_sm], popup='San Francisco').add_to(map_all)
folium.CircleMarker(location=[latitude_sf_sm, longitude_sf_sm], radius=10,
popup='San Francisco / SoMa Area', color='#3186cc', fill_color='#3186cc').add_to(map_all)

folium.Marker(location=[latitude_se_ps, longitude_se_ps], popup='Seattle').add_to(map_all)
folium.CircleMarker(location=[latitude_se_ps, longitude_se_ps], radius=10,
popup='Seattle / Pioneer Square', color='#3186cc', fill_color='#3186cc').add_to(map_all)

map_all

In [10]:
# Display map of the San Francisco / SoMa Area (South of Market Street)
map_sf_sm = folium.Map(location=[latitude_sf_sm, longitude_sf_sm], tiles='Stamen Terrain', zoom_start=12)

folium.Marker(location=[latitude_sf_sm, longitude_sf_sm], popup='San Francisco').add_to(map_sf_sm)
folium.CircleMarker(location=[latitude_sf_sm, longitude_sf_sm], radius=10,
popup='San Francisco / SoMa Area', color='#3186cc', fill_color='#3186cc').add_to(map_sf_sm)

map_sf_sm

In [11]:
CLIENT_ID = 'CIC0QBPUDADFCGUHR04XNX1N5ZKS15U51SUXSHXP5SVFZI4L'
CLIENT_SECRET = 'HQVQZB4L532EAGPGYZI4YCSHHNCZSSSFZ1BWVA2F1LIYN1BJ'
VERSION = 20210217

In [13]:

# San Francisco / SoMa area:
# Analyze venues / local businesses within 1500 meters of the target neighborhood (approx. 1 mile radius)
radius = 1500
limit = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude_sf_sm, longitude_sf_sm, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CIC0QBPUDADFCGUHR04XNX1N5ZKS15U51SUXSHXP5SVFZI4L&client_secret=HQVQZB4L532EAGPGYZI4YCSHHNCZSSSFZ1BWVA2F1LIYN1BJ&v=20210217&ll=37.775316,-122.419626&radius=1500&limit=100'

In [14]:
results = requests.get(url).json()

In [15]:
results

{'meta': {'code': 200, 'requestId': '602d4b6fd7e05c24796bf60b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Civic Center',
  'headerFullLocation': 'Civic Center, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 217,
  'suggestedBounds': {'ne': {'lat': 37.78881601350001,
    'lng': -122.40257832325781},
   'sw': {'lat': 37.76181598649998, 'lng': -122.43667367674217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50f21340e4b036c5cc0d7c7d',
       'name': 'SFJazz Center',
       'location': {'address': '201 Franklin St',
        'crossStreet': 'at Fell St',
        'lat': 37.77635001768186,
        'lng': 

In [16]:
for i in range(30):
    print(results['response']['groups'][0]['items'][i]['venue']['name'])

SFJazz Center
Blue Bottle Coffee
New Conservatory Theatre Center
Louise M. Davies Symphony Hall
Rich Table
Fatted Calf
Hotel Biron
Blue Bottle Coffee
Maker & Moss
Linden Room
Siam Orchid Traditional Thai Massage
20th Century Cafe
Linden Alley
San Francisco Wine & Cheese
Otoro Sushi
Zuni Café
Main Stage Of Davies Symphony Hall
War Memorial Opera House
Fitness SF
Arlequin Wine Merchant
San Francisco Ballet
Isotope Comics
Ritual Coffee Roasters
Chez Maman
Fig & Thistle Wine Bar
Dumpling Home
The Green Arcade
Boba Guys
Smitten Ice Cream
MAC Modern Appealing Clothing


### as same with the first CAPSTONE PROJECT:

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:

# Now we clean the JSON object and structure it into a pandas dataframe.
search_query_SF = 'venues'
venues_SF = results['response']['groups'][0]['items']
nearby_venues_SF = json_normalize(venues_SF)     # flatten JSON
nearby_venues_SF.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-50f21340e4b036c5cc0d7c7d-0,0,"[{'summary': 'This spot is popular', 'type': '...",50f21340e4b036c5cc0d7c7d,SFJazz Center,201 Franklin St,at Fell St,37.776350,-122.421539,"[{'label': 'display', 'lat': 37.77635001768186...",203,94102,US,Civic Center,San Francisco,CA,United States,"[201 Franklin St (at Fell St), San Francisco, ...","[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-5560dbdb498e91a2bcde84f6-1,0,"[{'summary': 'This spot is popular', 'type': '...",5560dbdb498e91a2bcde84f6,Blue Bottle Coffee,1355 Market St Ste 190,at 10th St,37.776286,-122.416867,"[{'label': 'display', 'lat': 37.77628641647586...",265,94103,US,NaN,San Francisco,CA,United States,"[1355 Market St Ste 190 (at 10th St), San Fran...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e-0-4a346043f964a520279c1fe3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4a346043f964a520279c1fe3,New Conservatory Theatre Center,25 Van Ness Ave,Oak St,37.775591,-122.419753,"[{'label': 'display', 'lat': 37.77559103972081...",32,94102,US,NaN,San Francisco,CA,United States,"[25 Van Ness Ave (Oak St), San Francisco, CA 9...","[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",0,[],75569151,NaN,NaN,NaN,NaN,NaN,NaN
3,e-0-4aa48566f964a520024720e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4aa48566f964a520024720e3,Louise M. Davies Symphony Hall,201 Van Ness Ave,btwn Grove & Hayes St,37.777976,-122.420157,"[{'label': 'display', 'lat': 37.7779761642689,...",299,94102,US,NaN,San Francisco,CA,United States,"[201 Van Ness Ave (btwn Grove & Hayes St), San...","[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e-0-500088f7d63e64b62bc19e6e-4,0,"[{'summary': 'This spot is popular', 'type': '...",500088f7d63e64b62bc19e6e,Rich Table,199 Gough St,at Oak St,37.774891,-122.422736,"[{'label': 'display', 'lat': 37.774891, 'lng':...",277,94102,US,NaN,San Francisco,CA,United States,"[199 Gough St (at Oak St), San Francisco, CA 9...","[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",0,[],52310779,NaN,NaN,NaN,NaN,NaN,NaN


# Repeat to Seattle/Pioneer Square data

In [19]:
map_se_ps = folium.Map(location=[latitude_se_ps, longitude_se_ps], tiles='Stamen Terrain', zoom_start=12)

folium.Marker(location=[latitude_se_ps, longitude_se_ps], popup='Seattle').add_to(map_se_ps)
folium.CircleMarker(location=[latitude_se_ps, longitude_se_ps], radius=10,
popup='Seattle / Pioneer Square', color='#3186cc', fill_color='#3186cc').add_to(map_se_ps)

map_se_ps

In [20]:
radius = 1500
limit = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude_se_ps, longitude_se_ps, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CIC0QBPUDADFCGUHR04XNX1N5ZKS15U51SUXSHXP5SVFZI4L&client_secret=HQVQZB4L532EAGPGYZI4YCSHHNCZSSSFZ1BWVA2F1LIYN1BJ&v=20210217&ll=47.601954,-122.329204&radius=1500&limit=100'

In [21]:
results = requests.get(url).json()

In [22]:
for i in range(30):
    print(results['response']['groups'][0]['items'][i]['venue']['name'])

Elm Coffee Roasters
Flatstick Pub
Smith Tower Observation Bar
Tsukushinbo
Columbia Tower Club
Cherry Street Public House
Maneki
Caffè Umbria
Fuji Bakery
Kinokuniya Book Store
Go Poké
Il Terrazzo Carmine
Uwajimaya
Zeitgeist Kunst & Kaffee
Metropolitan Grill
Pink Gorilla
Waterfall Garden Park
Courtyard by Marriott Seattle Downtown/Pioneer Square
Red Bowls
Loews Hotel 1000, Seattle
Cherry Street Coffee House
Dim Sum King
Kau Kau Barbeque Market
Matsu Sushi
Young Tea
Uwajimaya Deli
Chengdu Memory
Lumen Field
Wing Luke Museum of the Asian Pacific American Experience
Mike's Noodle House


# Now we clean the JSON object and structure it into a pd.DataFrame

In [23]:
search_query_SEA = 'venues'
venues_SEA = results['response']['groups'][0]['items']
# flatten the json
nearby_venues_SEA = json_normalize(venues_SEA)
nearby_venues_SEA.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.neighborhood,venue.venuePage.id
0,e-0-545803de498e7e758ac5605e-0,0,"[{'summary': 'This spot is popular', 'type': '...",545803de498e7e758ac5605e,Elm Coffee Roasters,240 2nd Avenue Ext S Ste 103,47.600152,-122.330944,"[{'label': 'display', 'lat': 47.60015237080675...",239,98104,US,Seattle,WA,United States,"[240 2nd Avenue Ext S Ste 103, Seattle, WA 981...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1553195,https://www.grubhub.com/restaurant/elm-coffee-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],NaN,NaN,NaN
1,e-0-56b101bb498e4905510a5798-1,0,"[{'summary': 'This spot is popular', 'type': '...",56b101bb498e4905510a5798,Flatstick Pub,240 2nd Ave S,47.600143,-122.331002,"[{'label': 'display', 'lat': 47.6001432, 'lng'...",242,98104,US,Seattle,WA,United States,"[240 2nd Ave S (Main), Seattle, WA 98104, Unit...","[{'id': '52e81612bcbc57f1066b79eb', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],Main,Pioneer Square,154127947
2,e-0-5ab5e04ce679bc6844b3448d-2,0,"[{'summary': 'This spot is popular', 'type': '...",5ab5e04ce679bc6844b3448d,Smith Tower Observation Bar,506 2nd Ave,47.602012,-122.331793,"[{'label': 'display', 'lat': 47.60201211476762...",194,98104,US,Seattle,WA,United States,"[506 2nd Ave, Seattle, WA 98104, United States]","[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,Pioneer Square,NaN
3,e-0-4a3be500f964a520e2a01fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4a3be500f964a520e2a01fe3,Tsukushinbo,515 S Main St,47.599963,-122.326878,"[{'label': 'display', 'lat': 47.59996315315612...",282,98104,US,Seattle,WA,United States,"[515 S Main St (btw 5th & 6th Ave S), Seattle,...","[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",2031600,https://www.grubhub.com/restaurant/tsukushinbo...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],btw 5th & 6th Ave S,NaN,NaN
4,e-0-451c049bf964a520803a1fe3-4,0,"[{'summary': 'This spot is popular', 'type': '...",451c049bf964a520803a1fe3,Columbia Tower Club,701 5th Ave Fl 75,47.604507,-122.330484,"[{'label': 'display', 'lat': 47.60450651693834...",299,98104,US,Seattle,WA,United States,"[701 5th Ave Fl 75, Seattle, WA 98104, United ...","[{'id': '52e81612bcbc57f1066b7a33', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN
